In [ ]:
!pip install streamlit pyngrok --quiet
!pip install apscheduler
!pip install watchdog
!pip install lightgbm shap seaborn scikit-learn joblib
!pip install pandas numpy scikit-learn lightgbm matplotlib
!apt-get update && apt-get install -y tshark




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,383 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/ma

In [ ]:
print("Columns in merged dataset:")
print(df.columns.tolist())


Columns in merged dataset:
['Filename', 'pslist.nproc', 'pslist.nppid', 'pslist.avg_threads', 'pslist.nprocs64bit', 'pslist.avg_handlers', 'dlllist.ndlls', 'dlllist.avg_dlls_per_proc', 'handles.nhandles', 'handles.avg_handles_per_proc', 'handles.nport', 'handles.nfile', 'handles.nevent', 'handles.ndesktop', 'handles.nkey', 'handles.nthread', 'handles.ndirectory', 'handles.nsemaphore', 'handles.ntimer', 'handles.nsection', 'handles.nmutant', 'ldrmodules.not_in_load', 'ldrmodules.not_in_init', 'ldrmodules.not_in_mem', 'ldrmodules.not_in_load_avg', 'ldrmodules.not_in_init_avg', 'ldrmodules.not_in_mem_avg', 'malfind.ninjections', 'malfind.commitCharge', 'malfind.protection', 'malfind.uniqueInjections', 'psxview.not_in_pslist', 'psxview.not_in_eprocess_pool', 'psxview.not_in_ethread_pool', 'psxview.not_in_pspcid_list', 'psxview.not_in_csrss_handles', 'psxview.not_in_session', 'psxview.not_in_deskthrd', 'psxview.not_in_pslist_false_avg', 'psxview.not_in_eprocess_pool_false_avg', 'psxview.not

In [ ]:
!tshark -r network_capture.pcap


Running as user "root" and group "root". This could be dangerous.
tshark: The file "network_capture.pcap" doesn't exist.


In [ ]:
!tshark -r network_capture.pcap -T fields -e ip.src -e ip.dst


Running as user "root" and group "root". This could be dangerous.
192.168.1.2	192.168.1.1
192.168.1.2	192.168.1.1
192.168.1.2	192.168.1.1
192.168.1.2	192.168.1.1
192.168.1.2	192.168.1.1


In [ ]:
!cat network_features.csv


In [ ]:
!pip install scapy --quiet

from scapy.all import Ether, IP, TCP, wrpcap

# Create 5 sample packets
packets = []
for i in range(5):
    pkt = Ether()/IP(dst="192.168.1.1", src="192.168.1.2")/TCP(dport=80, sport=1024+i)
    packets.append(pkt)

# Save to a PCAP file
pcap_filename = "network_capture.pcap"
wrpcap(pcap_filename, packets)

# Confirm file created
import os
print("PCAP created:", os.path.isfile(pcap_filename))
print("File size:", os.path.getsize(pcap_filename), "bytes")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.1 MB/s eta 0:00:00
PCAP created: True
File size: 374 bytes


In [ ]:
ls -lh /content/network_capture.pcap


-rw-r--r-- 1 root root 374 Apr  7 20:19 /content/network_capture.pcap


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import matplotlib
matplotlib.use('Agg')  # Use non-GUI backend
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib
import os
from datetime import datetime
import logging
import subprocess
from apscheduler.schedulers.background import BackgroundScheduler

# ----------------- Logging Setup -----------------
logging.basicConfig(filename='detection_system.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# ----------------- GUI Alert (Safe for Colab) -----------------
def show_ransomware_alert():
    try:
        import tkinter as tk
        root = tk.Tk()
        root.title("⚠️ ALERT: Suspicious Activity Detected ⚠️")
        root.configure(bg="red")
        root.attributes("-fullscreen", True)
        label = tk.Label(
            root,
            text="⚠️ WARNING: Potential Ransomware Activity Detected ⚠️\n\nPlease disconnect from the internet and shut down your system.",
            font=("Helvetica", 24, "bold"),
            bg="red",
            fg="white",
            wraplength=1000,
            justify="center"
        )
        label.pack(expand=True)
        root.mainloop()
    except Exception as e:
        print("⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]")

# ----------------- Safe CSV Loader -----------------
def safe_read_csv(filepath):
    try:
        return pd.read_csv(filepath)
    except UnicodeDecodeError:
        return pd.read_csv(filepath, encoding='ISO-8859-1')

# ----------------- Wireshark Feature Extraction -----------------
def extract_network_features(pcap_path, output_csv="network_features.csv"):
    tshark_command = [
        "tshark", "-r", pcap_path, "-T", "fields",
        "-e", "ip.src", "-e", "ip.dst",
        "-e", "tcp.srcport", "-e", "tcp.dstport",
        "-e", "udp.srcport", "-e", "udp.dstport",
        "-e", "_ws.col.Protocol",
        "-E", "header=y", "-E", "separator=,", "-E", "quote=d", "-E", "occurrence=f"
    ]
    try:
        with open(output_csv, "w") as out_csv:
            subprocess.run(tshark_command, stdout=out_csv, check=True)
        print(f"✅ Network features saved to {output_csv}")
    except Exception as e:
        print(f"[!] Error extracting network features: {e}")

# ----------------- Process Network CSV -----------------
def process_network_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
        features = {
            'total_packets': len(df),
            'unique_src_ips': df['ip.src'].nunique(),
            'unique_dst_ips': df['ip.dst'].nunique(),
            'tcp_connections': df['tcp.srcport'].notnull().sum(),
            'udp_connections': df['udp.srcport'].notnull().sum(),
        }
        protocol_counts = df['_ws.col.Protocol'].value_counts().to_dict()
        for proto, count in protocol_counts.items():
            features[f'proto_{proto.lower()}'] = count
        return pd.DataFrame([features])
    except Exception as e:
        print(f"[!] Failed to process network CSV: {e}")
        return pd.DataFrame()

# ----------------- Load & Label Datasets -----------------
df1 = safe_read_csv("/content/Output1.csv")
df2 = safe_read_csv("/content/output2.csv")
df3 = safe_read_csv("/content/output3.csv")
df1['Benign'] = 1
df2['Benign'] = 0
df3['Benign'] = 0

# ----------------- Optional Network Features -----------------
pcap_file_path = "/content/network_capture.pcap"
extract_network_features(pcap_file_path)
network_df = process_network_csv("network_features.csv")

if not network_df.empty:
    for col in network_df.columns:
        df1[col] = network_df[col].iloc[0]
        df2[col] = network_df[col].iloc[0]
        df3[col] = network_df[col].iloc[0]
else:
    print("⚠️ Network data not available, proceeding with static features.")

# ----------------- Merge and Preprocess -----------------
df = pd.concat([df1, df2, df3], ignore_index=True)
for col in ['FileName', 'Filename', 'md5Hash']:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)

X = df.drop('Benign', axis=1)
y = df['Benign']
logging.info(f"Label distribution:\n{y.value_counts()}")

X.fillna(X.median(), inplace=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ----------------- LightGBM Training -----------------
lgb_model = lgb.LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=10, num_leaves=64, random_state=42)
lgb_model.fit(X_train, y_train)

y_pred = lgb_model.predict(X_test)
y_proba = lgb_model.predict_proba(X_test)[:, 1]
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print(f"Accuracy: {acc * 100:.2f}% | ROC-AUC: {roc_auc:.2f}")

# ----------------- Save Artifacts -----------------
joblib.dump(lgb_model, "ransomware_model.pkl")
joblib.dump(X.columns.tolist(), "feature_columns.pkl")
joblib.dump(scaler, "scaler.pkl")

# ----------------- Confusion Matrix Plot -----------------
plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig("confusion_matrix.png")
plt.close()

# ----------------- ROC Curve -----------------
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC-AUC = {roc_auc:.2f}")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.savefig("roc_curve.png")
plt.close()

# ----------------- SHAP Explanation -----------------
explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_train)

shap_matrix = shap_values[1] if isinstance(shap_values, list) else shap_values
if len(shap_matrix.shape) == 1:
    shap_matrix = shap_matrix.reshape(-1, 1)

shap.summary_plot(shap_matrix, pd.DataFrame(X_train, columns=X.columns), show=False)
plt.savefig("shap_summary.png")
plt.close()

# ----------------- Cross-Validation -----------------
cv_score = cross_val_score(lgb_model, X_scaled, y, cv=5).mean()
print(f"5-Fold Cross-Validation Accuracy: {cv_score:.2f}")

# ----------------- Random Forest Comparison -----------------
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_acc = accuracy_score(y_test, rf.predict(X_test))
print(f"Random Forest Accuracy for Comparison: {rf_acc:.2f}")

# ----------------- Real-Time Simulation -----------------
def scan_file(features):
    model = joblib.load("ransomware_model.pkl")
    scaler = joblib.load("scaler.pkl")
    features_scaled = scaler.transform([features])
    prediction = model.predict(features_scaled)[0]
    return "Benign" if prediction == 1 else "Ransomware"

def isolate_file(file_name):
    print(f"[!] Threat detected in {file_name}. Isolating... (simulation)")
    logging.warning(f"Ransomware threat detected and isolated: {file_name}")
    show_ransomware_alert()

def log_event(file_name, status):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("scan_log.txt", "a") as log:
        log.write(f"{timestamp} | {file_name} | {status}\n")

# ----------------- Batch Scanning -----------------
def batch_scan(csv_path):
    df = safe_read_csv(csv_path)
    exclude_cols = ['FileName', 'Filename', 'md5Hash', 'Benign']

    feature_columns = joblib.load("feature_columns.pkl")
    scaler = joblib.load("scaler.pkl")
    model = joblib.load("ransomware_model.pkl")

    df_features = df[[col for col in feature_columns if col in df.columns]].copy()
    df_features = df_features.fillna(df_features.median())
    df_features = df_features.reindex(columns=feature_columns, fill_value=0)

    features_scaled = scaler.transform(df_features)
    predictions = model.predict(features_scaled)

    ransomware_detected = []

    with open("scan_log.txt", "a") as log:
        for idx, (fname, pred) in enumerate(zip(df.get('FileName', range(len(df))), predictions)):
            status = "Benign" if pred == 1 else "Malicious"
            if status == "Malicious":
                ransomware_detected.append((idx + 2, fname))  # +2 because header is line 1, then first row is 2
                show_ransomware_alert()
            log.write(f"{datetime.now()} - File: {fname}, Status: {status}\n")

    print("✅ Batch scan completed. See scan_log.txt for results.")

    # Display ransomware detections
    if ransomware_detected:
        print("\n🚨 Potential Ransomware Detected in the Following Files:")
        for line_num, file_name in ransomware_detected:
            print(f"🔴 Line {line_num}: {file_name}")
    else:
        print("\n✅ No ransomware detected in the scanned files.")


# ----------------- Auto Trigger PCAP Analysis -----------------
def auto_trigger_pcap_analysis():
    print("⏱️ Running scheduled PCAP analysis...")
    extract_network_features(pcap_file_path)
    batch_scan("/content/output3.csv")

scheduler = BackgroundScheduler()
scheduler.add_job(auto_trigger_pcap_analysis, 'interval', minutes=30)
scheduler.start()

# ----------------- Example Usage -----------------
batch_scan("/content/output3.csv")


✅ Network features saved to network_features.csv
[LightGBM] [Info] Number of positive: 119, number of negative: 561
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2721
[LightGBM] [Info] Number of data points in the train set: 680, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.175000 -> initscore=-1.550597
[LightGBM] [Info] Start training from score -1.550597
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/shap/explainers/_tree.py:544: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
<ipython-input-92-52727189705b>:170: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_matrix, pd.DataFrame(X_train, columns=X.columns), show=False)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecatio

[LightGBM] [Info] Number of positive: 120, number of negative: 560
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000557 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2999
[LightGBM] [Info] Number of data points in the train set: 680, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.176471 -> initscore=-1.540445
[LightGBM] [Info] Start training from score -1.540445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Random Forest Accuracy for Comparison: 0.85
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ransomware Detected! [GUI skipped in this environment]
⚠️ ALERT: Potential Ranso

In [ ]:
!apt-get install tshark -y
!pip install pyshark nest_asyncio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tshark is already the newest version (3.6.2-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!pip install volatility3


In [ ]:
!pip install watchdog psutil

Incorporate Behavioral Analysis (Dynamic Monitoring)

In [ ]:
import os
import time
import psutil
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from collections import defaultdict
import threading

# Suspicious extensions (common in ransomware attacks)
SUSPICIOUS_EXTENSIONS = [".locked", ".encrypted", ".cry", ".crypt", ".enc"]

# Monitor this directory (change if needed)
WATCHED_DIR = "./content/drive/MyDrive/television_detected"

# Create the folder if it doesn't exist
os.makedirs(WATCHED_DIR, exist_ok=True)

# A dictionary to keep track of file renaming events
rename_counter = defaultdict(int)
alert_triggered = False

class RansomwareEventHandler(FileSystemEventHandler):
    def on_moved(self, event):
        global alert_triggered
        if not event.is_directory:
            old_name = os.path.basename(event.src_path)
            new_name = os.path.basename(event.dest_path)
            if any(ext in new_name for ext in SUSPICIOUS_EXTENSIONS):
                rename_counter[time.time()] += 1
                print(f"[⚠️ ALERT] File renamed to suspicious extension: {new_name}")
                if rename_counter and not alert_triggered:
                    recent = sum(1 for t in rename_counter if time.time() - t < 30)
                    if recent >= 3:
                        alert_triggered = True
                        print("\n[🔥 POTENTIAL RANSOMWARE DETECTED] Multiple suspicious file renames detected in short time!\n")

def start_file_monitor():
    print(f"📂 Monitoring folder: {WATCHED_DIR}")
    event_handler = RansomwareEventHandler()
    observer = Observer()
    observer.schedule(event_handler, path=WATCHED_DIR, recursive=True)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

def monitor_system_resources():
    print("📊 Monitoring system resources for unusual spikes...")
    while True:
        cpu = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory().percent
        net = psutil.net_io_counters()
        if cpu > 80 or memory > 90:
            print(f"[⚠️ HIGH USAGE] CPU: {cpu}%, Memory: {memory}%")
        time.sleep(5)

# Run file and system monitor in parallel
if __name__ == "__main__":
    file_thread = threading.Thread(target=start_file_monitor)
    sys_thread = threading.Thread(target=monitor_system_resources)

    file_thread.start()
    sys_thread.start()

    file_thread.join()
    sys_thread.join()


📂 Monitoring folder: ./content/drive/MyDrive/television_detected
📊 Monitoring system resources for unusual spikes...
[⚠️ HIGH USAGE] CPU: 96.0%, Memory: 10.9%
[⚠️ HIGH USAGE] CPU: 95.5%, Memory: 12.5%
[⚠️ HIGH USAGE] CPU: 83.8%, Memory: 13.6%
[⚠️ HIGH USAGE] CPU: 100.0%, Memory: 14.1%


KeyboardInterrupt: 

In [ ]:
import os
import json
import pandas as pd
import joblib
import subprocess

memory_file = 'sample.raw'  # Your memory dump file

# Updated function to run Volatility3
def run_volatility(plugin, mem_file):
    cmd = [
        "python3", "volatility3/vol.py",
        "-f", mem_file,
        f"windows.{plugin}",
        "--output", "json"
    ]
    output = subprocess.check_output(cmd, universal_newlines=True)
    return json.loads(output)

# 1. Extract features from each plugin
def extract_features(mem_file):
    features = {}

    # pslist
    pslist = run_volatility('pslist.PsList', mem_file)
    features['pslist.nproc'] = len(pslist)
    features['pslist.avg_threads'] = sum([proc.get('Threads', 0) for proc in pslist]) / len(pslist) if pslist else 0

    # handles
    handles = run_volatility('handles.Handles', mem_file)
    features['handles.nhandles'] = len(handles)
    features['handles.nport'] = sum(1 for h in handles if h.get('Type') == 'Port')
    features['handles.nfile'] = sum(1 for h in handles if h.get('Type') == 'File')
    # You can extend this for other handle types...

    # dlllist
    dlllist = run_volatility('dlllist.DllList', mem_file)
    features['dlllist.ndlls'] = len(dlllist)
    unique_pids = set([d.get('Pid') for d in dlllist if 'Pid' in d])
    features['dlllist.avg_dlls_per_proc'] = len(dlllist) / len(unique_pids) if unique_pids else 0

    # malfind
    malfind = run_volatility('malfind.Malfind', mem_file)
    features['malfind.ninjections'] = len(malfind)
    features['malfind.uniqueInjections'] = len(set([m.get('Pid') for m in malfind if 'Pid' in m]))

    # svcscan
    svcscan = run_volatility('svcscan.SvcScan', mem_file)
    features['svcscan.nservices'] = len(svcscan)
    features['svcscan.nactive'] = sum(1 for s in svcscan if s.get('State') == 'SERVICE_RUNNING')

    # callbacks
    callbacks = run_volatility('callbacks.Callbacks', mem_file)
    features['callbacks.ncallbacks'] = len(callbacks)

    # Extend with more plugins as needed...

    return features

# 2. Run on memory sample
features = extract_features(memory_file)
df = pd.DataFrame([features])

# 3. Save extracted features to CSV
df.to_csv('extracted_features.csv', index=False)

# 4. Load model and make prediction
model = joblib.load('ransomware_model.pkl')
predictions = model.predict(df)
df['ransomware_detected'] = predictions

# 5. Save final output with prediction
df.to_csv('prediction_output.csv', index=False)

# 6. Print result
print(df[['ransomware_detected']])


CalledProcessError: Command '['python3', 'volatility3/vol.py', '-f', 'sample.raw', 'windows.pslist.PsList', '--output', 'json']' returned non-zero exit status 2.

In [ ]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
import shutil

WATCH_FOLDER = r"C:\Users\Sanket Chavhan\Desktop\Sem6\ISS\ISS Project"  # Folder to watch for new files

class RansomwareMonitorHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory:
            filepath = event.src_path
            print(f"\n📂 New file detected: {filepath}")

            # Simulate feature extraction for a new file
            # (In real use, you'd extract meaningful features here)
            try:
                df = pd.read_csv(filepath)
                feature_columns = pickle.load(open('feature_columns.pkl', 'rb'))
                scaler = pickle.load(open('scaler.pkl', 'rb'))
                model = pickle.load(open('ransomware_model.pkl', 'rb'))

                df.fillna(0, inplace=True)
                for col in feature_columns:
                    if col not in df.columns:
                        df[col] = 0
                df = df[feature_columns]
                df_scaled = scaler.transform(df)

                predictions = model.predict(df_scaled)
                for i, pred in enumerate(predictions):
                    label = 'Ransomware' if pred == 0 else 'Benign'
                    print(f"🧪 File Scan Result: {label}")

                    log_event(filepath, label)

                    if label == "Ransomware":
                        isolate_file(filepath)
                        send_email_alert("New Ransomware File Detected", f"File: {filepath} detected as ransomware.")
            except Exception as e:
                print(f"❌ Error processing file: {e}")

def start_file_monitoring():
    os.makedirs(WATCH_FOLDER, exist_ok=True)
    print(f"👀 Watching folder for new files: {WATCH_FOLDER}")
    event_handler = RansomwareMonitorHandler()
    observer = Observer()
    observer.schedule(event_handler, path=WATCH_FOLDER, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


In [ ]:
print("Columns in the combined dataset:", df.columns.tolist())


Columns in the combined dataset: ['pslist.nproc', 'pslist.nppid', 'pslist.avg_threads', 'pslist.nprocs64bit', 'pslist.avg_handlers', 'dlllist.ndlls', 'dlllist.avg_dlls_per_proc', 'handles.nhandles', 'handles.avg_handles_per_proc', 'handles.nport', 'handles.nfile', 'handles.nevent', 'handles.ndesktop', 'handles.nkey', 'handles.nthread', 'handles.ndirectory', 'handles.nsemaphore', 'handles.ntimer', 'handles.nsection', 'handles.nmutant', 'ldrmodules.not_in_load', 'ldrmodules.not_in_init', 'ldrmodules.not_in_mem', 'ldrmodules.not_in_load_avg', 'ldrmodules.not_in_init_avg', 'ldrmodules.not_in_mem_avg', 'malfind.ninjections', 'malfind.commitCharge', 'malfind.protection', 'malfind.uniqueInjections', 'psxview.not_in_pslist', 'psxview.not_in_eprocess_pool', 'psxview.not_in_ethread_pool', 'psxview.not_in_pspcid_list', 'psxview.not_in_csrss_handles', 'psxview.not_in_session', 'psxview.not_in_deskthrd', 'psxview.not_in_pslist_false_avg', 'psxview.not_in_eprocess_pool_false_avg', 'psxview.not_in_et

In [ ]:
app_code = """
import streamlit as st
import pandas as pd
import joblib
from datetime import datetime
import os

st.set_page_config(page_title="AI-Powered Ransomware Detection", layout="wide")
st.title("🔐 AI-Powered Ransomware Detection System")

st.sidebar.title("Navigation")
choice = st.sidebar.radio("Go to", ["Home", "Scan File", "Batch Scan Log"])

model = joblib.load("ransomware_model.pkl")

def predict(features):
    return "Benign" if model.predict([features])[0] == 1 else "Ransomware"

def log_event(file_name, status):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("scan_log.txt", "a") as log:
        log.write(f"{timestamp} | {file_name} | {status}\\n")

if choice == "Home":
    st.write("Welcome to the intelligent ransomware detection system.")
    st.image("confusion_matrix.png", caption="Confusion Matrix", use_column_width=True)
    st.image("roc_curve.png", caption="ROC Curve", use_column_width=True)
    st.image("shap_summary.png", caption="Feature Importance (SHAP)", use_column_width=True)

elif choice == "Scan File":
    st.header("🛡️ Upload File Features to Scan")
    uploaded = st.file_uploader("Upload a CSV with a single row of file features (excluding FileName, md5Hash)", type=["csv"])

    if uploaded:
        input_df = pd.read_csv(uploaded)
        prediction = predict(input_df.iloc[0])
        st.write(f"### 🧠 Prediction: `{prediction}`")
        log_event("Uploaded File", prediction)

elif choice == "Batch Scan Log":
    st.header("📄 Previous Batch Scan Results")
    if os.path.exists("scan_log.txt"):
        with open("scan_log.txt", "r") as f:
            logs = f.readlines()
        st.text("".join(logs[-20:]))
    else:
        st.warning("No logs found yet.")
"""

# Save to file
with open("app.py", "w") as f:
    f.write(app_code)


In [ ]:
!ngrok config add-authtoken 2vPgtxYFAbWgFn43F6jMgoUMeOf_2PZP1kMwRKib2JwU7JKuc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# 1. Install everything if not done yet
!pip install pyngrok streamlit -q

# 2. Kill any existing ngrok sessions
from pyngrok import ngrok
ngrok.kill()

# 3. Run your streamlit app (replace with your actual app filename)
!streamlit run app.py &  # Replace with your actual file

# 4. Connect ngrok
import time
time.sleep(5)  # Give Streamlit some time to start
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at 👉 {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.72.117.226:8501

  Stopping...
Streamlit app is live at 👉 NgrokTunnel: "https://61c3-34-72-117-226.ngrok-free.app" -> "http://localhost:8501"
